## [--- Day 9: Movie Theater ---](https://adventofcode.com/2025/day/9)

In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from typing import ClassVar, Iterator


@dataclass(frozen=True, slots=True, order=True)
class vector2:
    """Crude 2D vector wrapper for tuples"""

    x: int = 0
    y: int = 0

    zero: ClassVar["vector2"]
    up: ClassVar["vector2"]
    down: ClassVar["vector2"]
    left: ClassVar["vector2"]
    right: ClassVar["vector2"]

    def rotated_left(self) -> "vector2":
        return vector2(self.y, -self.x)

    def rotated_right(self) -> "vector2":
        return vector2(-self.y, self.x)

    def reflected_around(self, origin: "vector2") -> "vector2":
        return vector2(-self.x + 2 * origin.x, -self.y + 2 * origin.y)

    def distance_to(self, other: "vector2") -> int:
        return abs(self.x - other.x) + abs(self.y - other.y)

    def rect_area(self, other: "vector2") -> int:
        return (abs(self.x - other.x) + 1) * (abs(self.y - other.y) + 1)

    def __iter__(self) -> Iterator[int]:
        yield self.x
        yield self.y

    def __add__(self, other: "vector2") -> "vector2":
        return vector2(self.x + other.x, self.y + other.y)

    def __sub__(self, other: "vector2") -> "vector2":
        return vector2(self.x - other.x, self.y - other.y)

    def __repr__(self) -> str:
        return f"({self.x}, {self.y})"


vector2.zero = vector2(0, 0)
vector2.up = vector2(0, -1)
vector2.down = vector2(0, 1)
vector2.left = vector2(-1, 0)
vector2.right = vector2(1, 0)


from math import inf


class Grid:
    def __init__(self, data: list[vector2]) -> None:
        self.data = data

        xs = [p.x for p in self.data]
        ys = [p.y for p in self.data]
        self.min_x, self.max_x = min(xs), max(xs)
        self.min_y, self.max_y = min(ys), max(ys)

        self.width = self.max_x - self.min_x + 1
        self.height = self.max_y - self.min_y + 1

        # base: list[list[str]] = [['.'] * self.width for _ in range(self.height)]

        # for pos in data:
        #     base[pos.y - self.min_y][pos.x - self.min_x] = '#'

        # self.base = base

    def in_bounds(self, pos: vector2) -> bool:
        return self.min_x <= pos.x <= self.max_x and self.min_y <= pos.y <= self.max_y

    def rect_area(self, a: vector2, b: vector2) -> int:
        return (abs(a.x - b.x) + 1) * (abs(a.y - b.y) + 1)

    # def draw_rect(self, a: vector2, b: vector2, char: str = 'O') -> None:
    #     grid = [row.copy() for row in self.base]

    #     for y in range(min(a.y, b.y), max(a.y, b.y) + 1):
    #         for x in range(min(a.x, b.x), max(a.x, b.x) + 1):
    #             grid[y - self.min_y][x - self.min_x] = char

    #     print('\n'.join(["".join(row) for row in grid]))

    # def __getitem__(self, pos: vector2|tuple[int, int]) -> str:
    #     if self.in_bounds(pos):
    #         return self.base[pos[1] - self.min_y][pos[0] - self.min_x]

    # def __setitem__(self, pos: vector2|tuple[int, int], value: str) -> None:
    #     if self.in_bounds(pos):
    #         self.base[pos[1] - self.min_y][pos[0] - self.min_x] = value

    # def __repr__(self) -> str:
    #     return "\n".join("".join(row) for row in self.base)

In [9]:
def solve() -> None:
    with open("..\\data\\09.txt") as file:
        coords = [line.strip().split(",") for line in file.readlines()]

    grid = Grid([vector2(int(x), int(y)) for x, y in coords])

    # Horizons
    north_east: list[vector2] = []
    south_east: list[vector2] = []
    south_west: list[vector2] = []
    north_west: list[vector2] = []

    # Lexicographical ascending sort, x then y
    grid.data.sort()
    max_y: int = 0
    for pos in grid.data:
        if pos.y > max_y:
            north_east.append(pos)
            max_y = pos.y

    min_y: int = grid.height
    for pos in reversed(grid.data):
        if pos.y < min_y:
            south_west.append(pos)
            min_y = pos.y

    # Lexicographical sort, x descending then y ascending
    grid.data.sort(key=lambda pos: (-pos.x, pos.y))
    max_y = 0
    for pos in grid.data:
        if pos.y > max_y:
            north_west.append(pos)
            max_y = pos.y

    min_y = grid.height
    for pos in reversed(grid.data):
        if pos.y < min_y:
            south_east.append(pos)
            min_y = pos.y

    non_dominated_points = set(north_east + south_east + south_west + north_west)
    dominated_points = set(grid.data) - non_dominated_points

    # for point in dominated_points:
    #     grid[point] = 'x'

    print(f"{len(dominated_points)} dominated points removed")
    # print(grid)
    print(f"{len(non_dominated_points)} non-dominated points remain\n")

    ne_sw_pair: tuple[vector2, vector2] = (vector2.zero, vector2.zero)
    ne_sw_area: int = 0

    for ne in north_east:
        for sw in south_west:
            area = ne.rect_area(sw)
            if area > ne_sw_area:
                ne_sw_area = area
                ne_sw_pair = (ne, sw)

    print(f"NE-SW Max Area: {ne_sw_area} between {ne_sw_pair[0]} and {ne_sw_pair[1]}")
    # grid.draw_rect(ne_sw_pair[0], ne_sw_pair[1])

    se_nw_pair: tuple[vector2, vector2] = (vector2.zero, vector2.zero)
    se_nw_area: int = 0

    for se in south_east:
        for nw in north_west:
            area = se.rect_area(nw)
            if area > se_nw_area:
                se_nw_area = area
                se_nw_pair = (se, nw)

    print(f"SE-NW Max Area: {se_nw_area} between {se_nw_pair[0]} and {se_nw_pair[1]}")
    # grid.draw_rect(se_nw_pair[0], se_nw_pair[1])

    print(f"The largest area between any two points is {max(ne_sw_area, se_nw_area)}")


solve()

280 dominated points removed
216 non-dominated points remain

NE-SW Max Area: 4678664144 between (16361, 84880) and (83532, 15229)
SE-NW Max Area: 4761736832 between (17804, 13266) and (86507, 82573)
The largest area between any two points is 4761736832


In [10]:
class Compressed_Grid:
    def __init__(self, data: list[vector2]) -> None:
        self.points = data
        self.xs = sorted(list(set(x for x, _ in data)))
        self.ys = sorted(list(set(y for _, y in data)))
        self.x_map: dict[int, int] = {x: i for i, x in enumerate(self.xs)}
        self.y_map: dict[int, int] = {y: i for i, y in enumerate(self.ys)}

        self.width = len(self.xs) - 1
        self.height = len(self.ys) - 1
        self.data: list[list[bool]] = [[False] * self.width for _ in range(self.height)]
        vertical_boundaries: list[list[bool]] = [
            [False] * self.width for _ in range(self.height)
        ]

        for i in range(len(self.points)):
            p1 = self.points[i]
            p2 = self.points[(i + 1) % len(self.points)]

            # Draw line from p1 to p2
            if p1.x == p2.x:  # Vertical line
                x = self.x_map[p1.x]
                if x >= self.width:
                    continue

                y_start = min(self.y_map[p1.y], self.y_map[p2.y])
                y_end = max(self.y_map[p1.y], self.y_map[p2.y])

                for y in range(y_start, y_end):
                    vertical_boundaries[y][x] = not vertical_boundaries[y][x]

        for y in range(self.height):
            is_inside = False
            for x in range(self.width):
                if vertical_boundaries[y][x]:
                    is_inside = not is_inside

                self.data[y][x] = is_inside

    def in_bounds(self, pos: vector2) -> bool:
        return pos.x in self.x_map and pos.y in self.y_map

    def rect_area(self, a: vector2, b: vector2) -> int:
        return (abs(a.x - b.x) + 1) * (abs(a.y - b.y) + 1)

    def is_filled(self, a: vector2, b: vector2) -> bool:
        x_start = min(self.x_map[a.x], self.x_map[b.x])
        x_end = max(self.x_map[a.x], self.x_map[b.x])
        y_start = min(self.y_map[a.y], self.y_map[b.y])
        y_end = max(self.y_map[a.y], self.y_map[b.y])

        for y in range(y_start, y_end):
            for x in range(x_start, x_end):
                if self.data[y][x]:
                    continue
                return False

        return True

    def __getitem__(self, pos: vector2) -> bool:
        return self.data[pos.y][pos.x] if self.in_bounds(pos) else False

    def __setitem__(self, pos: vector2, value: bool) -> None:
        if self.in_bounds(pos):
            self.data[pos.y][pos.x] = value


def part_2() -> None:
    with open("..\\data\\09.txt") as file:
        data: list[list[str]] = [line.strip().split(",") for line in file.readlines()]

    points: list[vector2] = [vector2(int(x), int(y)) for x, y in data]
    polygon = Compressed_Grid(points)

    max_area: int = 0
    max_pair: tuple[vector2, vector2] = (vector2.zero, vector2.zero)
    for i, p1 in enumerate(points):
        for p2 in points[i + 1 :]:
            if polygon.is_filled(p1, p2):
                area = polygon.rect_area(p1, p2)
                if area <= max_area:
                    continue

                max_area = area
                max_pair = (p1, p2)

    print(
        f"Part 2: Max filled area is {max_area} between {max_pair[0]} and {max_pair[1]}"
    )


part_2()

Part 2: Max filled area is 1452422268 between (4615, 66437) and (94737, 50322)


In [ ]:
# TODO: convert the grid into an image so it's more efficient to visualize large grids.
# TODO: implement prefix sum for fast area queries

In [ ]:
import timeit
from collections import namedtuple
from dataclasses import dataclass


# 1. Your current approach
class VectorProperty(tuple):
    @property
    def x(self):
        return self[0]

    @property
    def y(self):
        return self[1]

    def rect_area(self, other):
        return (abs(self.x - other.x) + 1) * (abs(self.y - other.y) + 1)

    def rotated_left(self) -> "VectorProperty":
        return VectorProperty((self.y, -self.x))


# 2. Namedtuple
VectorNamed = namedtuple("VectorNamed", ["x", "y"])


def area_named(a, b):
    return (abs(a.x - b.x) + 1) * (abs(a.y - b.y) + 1)


def rotated_left_named(v: VectorNamed) -> VectorNamed:
    return VectorNamed(v.y, -v.x)


# 3. Dataclass with slots (Standard for high performance)
@dataclass(slots=True, frozen=True)
class VectorSlot_1:
    x: int
    y: int

    def rect_area(self, other):
        return (abs(self.x - other.x) + 1) * (abs(self.y - other.y) + 1)

    def rotated_left(self) -> "VectorSlot_1":
        return VectorSlot_1(self.y, -self.x)


@dataclass(slots=True, frozen=True)
class VectorSlot_2:
    x: int
    y: int

    def rect_area(self, other):
        return (abs(self.x - other.x) + 1) * (abs(self.y - other.y) + 1)

    def rotated_left(self) -> "VectorSlot_2":
        return type(self)(self.y, -self.x)


# Test Data
v1_prop, v2_prop = VectorProperty((10, 20)), VectorProperty((50, 60))
v1_named, v2_named = VectorNamed(10, 20), VectorNamed(50, 60)
v1_slot, v2_slot = VectorSlot_1(10, 20), VectorSlot_1(50, 60)
v1_slot2, v2_slot2 = VectorSlot_2(10, 20), VectorSlot_2(50, 60)

print(
    f"Calculating the area between {v1_prop} and {v2_prop}:\nArea = {v1_prop.rect_area(v2_prop)}\n"
)

# Benchmarking
iters = 1_000_000
t_prop = timeit.timeit(lambda: v1_prop.rotated_left(), number=iters)
t_named = timeit.timeit(lambda: rotated_left_named(v1_named), number=iters)
t_slot = timeit.timeit(lambda: v1_slot.rotated_left(), number=iters)
t_slot_2 = timeit.timeit(lambda: v1_slot2.rotated_left(), number=iters)

print(f"Property:   {t_prop:.4f}s")
print(f"NamedTuple: {t_named:.4f}s")
print(f"Slots:      {t_slot:.4f}s")
print(f"Slots 2:    {t_slot_2:.4f}s")
print(f"Delta:      {t_slot - t_slot_2:.4f}s")
print(f"Perc.Delta: {((t_slot - t_slot_2) / t_slot) * 100:.2f}%")

Calculating the area between (10, 20) and (50, 60):
Area = 1681

Property:   0.3406s
NamedTuple: 0.3857s
Slots:      0.4832s
Slots 2:    0.5121s
Delta:      -0.0288s
Perc.Delta: -5.97%
